# Scoring with Managed Online Endpoints

In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<Subscription ID>"
    resource_group = "<ResourceGroup Name>"
    workspace = "<<WorkspaceName>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

We could not find config.json in: . or in its parent directories. 
MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x7f87d8cda820>,
         subscription_id=25758354-2fe7-426a-be0e-1ad20058340d,
         resource_group_name=azureml-live,
         workspace_name=ftaliveazureml)


In [2]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes

In [3]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="this is a online endpoint for diabetes LGBM mlflow model",
    auth_mode="key",
    tags={"model": "diabetes"},
)

ml_client.online_endpoints.begin_create_or_update(endpoint)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-06220908067978.southeastasia.inference.ml.azure.com/score', 'swagger_uri': 'https://endpoint-06220908067978.southeastasia.inference.ml.azure.com/swagger.json', 'name': 'endpoint-06220908067978', 'description': 'this is a online endpoint for diabetes LGBM mlflow model', 'tags': {'model': 'diabetes'}, 'properties': {'azureml.onlineendpointid': '/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/resourcegroups/azureml-live/providers/microsoft.machinelearningservices/workspaces/ftaliveazureml/onlineendpoints/endpoint-06220908067978', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/25758354-2fe7-426a-be0e-1ad20058340d/providers/Microsoft.MachineLearningServices/locations/southeastasia/mfeOperationsStatus/oe:b417752f-200a-46d5-b47e-800a43295269:72a84348-8d04-4f47-a12d-fa11e8d996d7?api-version=2022-02-01-preview'}, 'id': '/subscriptions/25758354

In [4]:
# create a blue deployment

#For MLFlow models, scoring and environment files are auto-generated

model = ml_client.models.get(name="Diabetest-LGBM-Model",version="1")

blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)

ml_client.online_deployments.begin_create_or_update(blue_deployment)

Check: endpoint endpoint-06220908067978 exists
Creating/updating online deployment blue 

.............................................................................................

Done (8m 11s)


In [5]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name='endpoint-06210618832738',
    deployment_name="blue",
    request_file="./050_scripts/online/sample_request.json",
)

'[227.92313383298713, 227.92313383298713]'

In [6]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name='endpoint-06210618832738')

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'blue': 0}
https://endpoint-06210618832738.southeastasia.inference.ml.azure.com/score
